In [1]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JLD2, FileIO
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [2]:
include("../helper_functions.jl");

In [3]:
system = MacroEnergy.load_system(@__DIR__)
model = MacroEnergy.generate_model(system)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.31 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ W

A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 788452
├ num_constraints: 1848444
│ ├ AffExpr in MOI.EqualTo{Float64}: 455523
│ ├ AffExpr in MOI.GreaterThan{Float64}: 78840
│ ├ AffExpr in MOI.LessThan{Float64}: 543149
│ ├ VariableRef in MOI.EqualTo{Float64}: 13
│ └ VariableRef in MOI.GreaterThan{Float64}: 770919
└ Names registered in the model
  └ :cRetrofitCapacity, :eFixedCost, :eVariableCost, :vREF

In [4]:
MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 1077512 rows, 788452 columns and 3345291 nonzeros
Model fingerprint: 0x42aec24c
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 605002 rows and 400236 columns
Presolve time: 0.64s
Presolved: 472510 rows, 388216 columns, 2274912 nonzeros
Ordering t

In [5]:
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results/trad_oxy_MEA")
mkpath(results_dir)
save_results(results_dir, case, system)

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/results/trad_oxy_MEA/0pct_co2_reduction_flows.csv"

In [ ]:
# Run model for different CO2 caps, if you have a no_cap_co2_level

no_cap_co2_level = 1.603635558784975e6
co2_reductions = [0.2, 0.4, 0.6, 0.8, 0.95] # If there is a 0, it must be the first value
system = nothing
model = nothing

for co2_reduction in co2_reductions
    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = MacroEnergy.load_system(case_path)

    # Get the CO2 node
    nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]
    co2_node = MacroEnergy.get_nodes_sametype(nodes, CO2)[1] # the cement_co2_sink node is the first CO2 node

    if co2_reduction == 0 # If there is no CO2 cap
        case = "0pct_co2_reduction"
        co2_node.constraints = AbstractTypeConstraint[] # Remove CO2 cap constraint
    else # If there is not a CO2 cap
        case = string(floor(Int, co2_reduction * 100)) * "pct_co2_reduction"
        co2_node.rhs_policy[CO2CapConstraint] = no_cap_co2_level * (1 - co2_reduction) # Set CO2 cap
        print(no_cap_co2_level * (1 - co2_reduction))
    end

    # Run model
    model = MacroEnergy.generate_model(system)
    MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
    MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    MacroEnergy.optimize!(model)

    # Save results
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)
    save_results(results_dir, case, system)
    # @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    # println("wrote results")

    # Record the no_cap_co2_level for next iterations
    if co2_reduction == 0
        no_cap_co2_level = MacroEnergy.value(sum(co2_node.operation_expr[:emissions]))
    end
end

###### ###### ######
Running case at /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test


┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ Wa

1.2829084470279801e6

┌ Info: Done loading system. It took 0.35 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:25
┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 7.29 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 1077514 rows, 788453 columns and 3424133 nonzeros
Model fingerprint: 0x798f3062
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+06]
Presolve removed 605003 rows and 400237 columns
Presolve time: 0.63s
Presolved: 472511 rows, 388216 columns, 2353752 nonzeros
Ordering t

...

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ Wa

0.0

┌ Info: Done loading system. It took 0.32 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:25
┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 7.63 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 1077514 rows, 788453 columns and 3424133 nonzeros
Model fingerprint: 0x3469d531
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 561204 rows and 377242 columns
Presolve time: 0.08s

Barrier solved model in 0 iterations and 0.08 seconds (0.26 work u

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.